___

<a href='https://sites.google.com/fat.uerj.br/livia'><img src="../figures/capa2.png"/></a>
___
___

# Introdução

"A união faz a força". Este antigo ditado expressa de maneira bastante eficaz a ideia subjacente que rege os poderosos métodos de "ensemble" em aprendizado de máquina. De maneira geral, os métodos de aprendizado em conjunto, frequentemente confiáveis nos primeiros lugares de diversas competições de aprendizado de máquina (incluindo as competições do Kaggle), baseiam-se na hipótese de que a combinação de vários modelos pode frequentemente resultar em um modelo muito mais poderoso.

Neste tutorial, você aprenderá como implementar e aplicar os seguintes métodos de aprendizado em conjunto em problemas de aprendizado de máquina:

* O método de "bagging" (bootstrap aggregating);
* O método de "boosting";
* O método de "stacking".

Para começar, vamos primeiro definir o que é um método de aprendizado em conjunto.

## Método de aprendizado em conjunto - *Ensemble Learning*?

Um método de aprendizado em conjunto é um método que combina vários algoritmos de aprendizado de máquina (*weak learners* - aprendizes fracos) para obter um modelo preditivo mais poderoso. Um método de aprendizado em conjunto pode, portanto, ser considerado como um meta-algoritmo, ou seja, um algoritmo que combina vários algoritmos de aprendizado de máquina em um modelo preditivo. Existem dois tipos principais de métodos de aprendizado em conjunto: os métodos de "bagging", os métodos de "boosting" e o métodos de "stacking", que veremos em detalhes nas próximas seções.

## Único aprendiz fraco
Em aprendizado de máquina, seja enfrentando um problema de classificação ou de regressão, a escolha do modelo é extremamente importante para ter qualquer chance de obter bons resultados. Essa escolha pode depender de muitas variáveis do problema: quantidade de dados, dimensionalidade do espaço, hipóteses de distribuição...

Um baixo viés (*bias*) e uma baixa variância (*variance*), embora geralmente variem em direções opostas, são as duas características mais fundamentais esperadas para um modelo. De fato, para ser capaz de "resolver" um problema, queremos que nosso modelo tenha graus de liberdade suficientes para resolver a complexidade subjacente dos dados com os quais estamos trabalhando, mas também queremos que não tenha muitos graus de liberdade para evitar alta variância e ser mais robusto. Isso é conhecido como o trade-off viés-variância.

<figure>
<img src="../figures/tradeoff.webp" style="width:80%">
<figcaption align = "center"></figcaption>
</figure>



Na teoria de aprendizado em conjunto, chamamos de "aprendizes fracos" (ou modelos base) os modelos que podem ser usados como blocos de construção para projetar modelos mais complexos, combinando vários deles. Na maioria das vezes, esses modelos básicos não têm um desempenho tão bom por si mesmos, seja porque têm um viés elevado (modelos com baixo grau de liberdade, por exemplo) ou porque têm muita variância para serem robustos (modelos com alto grau de liberdade, por exemplo).

## Combinação de aprendizes fracos
Para estabelecer um método de aprendizado em conjunto, primeiro precisamos selecionar nossos modelos bases a serem agregados. Na maioria das vezes (incluindo nos métodos conhecidos de bagging e boosting), é utilizado um único algoritmo de aprendizado base, de modo que tenhamos aprendizes fracos homogêneos treinados de maneiras diferentes. O modelo em conjunto que obtemos é então chamado de **homogêneo**. No entanto, existem também métodos que utilizam diferentes tipos de algoritmos de aprendizado base: alguns aprendizes fracos heterogêneos são combinados em um modelo de **conjunto heterogêneo**.

Um ponto importante é que a escolha de nossos aprendizes fracos deve ser coerente com a forma como agregamos esses modelos. Se escolhermos modelos base com baixo viés mas alta variância, deve ser com um método de agregação que tenda a reduzir a variância, enquanto se escolhermos modelos base com baixa variância mas alto viés, deve ser com um método de agregação que tenda a reduzir o viés.

Isso nos leva à questão de como combinar esses modelos. Podemos mencionar três tipos principais de meta-algoritmos que visam combinar aprendizes fracos:

- **Bagging:** muitas vezes considera aprendizes fracos homogêneos, aprendendo-os de forma independente uns dos outros **em paralelo** e combinando-os seguindo algum tipo de processo de média determinístico.

- **Boosting:** muitas vezes considera aprendizes fracos homogêneos, aprendendo-os **sequencialmente** de maneira muito adaptativa (um modelo base depende dos anteriores) e combinando-os seguindo uma estratégia determinística.

- **Stacking:** muitas vezes considera aprendizes fracos **heterogêneos**, aprendendo-os em paralelo e combinando-os treinando um meta-modelo para gerar uma previsão com base nas diferentes previsões dos modelos fracos.

De maneira geral, podemos dizer que o bagging se concentra principalmente em obter um modelo em conjunto com menos variância do que seus componentes, enquanto boosting e stacking tentarão principalmente produzir modelos fortes com menos viés do que seus componentes (mesmo que a variância também possa ser reduzida).

# Bagging

O bagging é um método de aprendizado em conjunto que visa reduzir a variância de um modelo em conjunto, combinando vários modelos em um modelo preditivo. O bagging é uma abreviação de *bootstrap aggregating*, que pode ser traduzido como agregação de amostragem aleatória. Já vimos o conceito de bootstraping superficialmente no tutorial sobre estatistica inferencial. 

## Bootstraping

O bootstraping é um método de reamostragem com reposição. A ideia é que, a partir de um conjunto de dados de tamanho $D$, podemos gerar um novo conjunto de dados de tamanho $d$ amostrando aleatoriamente $d$ pontos do conjunto de dados original com reposição. Isso significa que um ponto pode ser amostrado várias vezes e outros podem não ser amostrados. O bootstraping é uma técnica muito útil para estimar a distribuição de um estimador, como a média ou a variância de um conjunto de dados.


<figure>
<img src="../figures/bootstrap.webp" style="width:80%">
<figcaption align = "center"></figcaption>
</figure>

## Como funciona o bagging?

Ao treinar um modelo, seja para um problema de classificação ou regressão, obtemos uma função que recebe uma entrada, retorna uma saída e é definida em relação ao conjunto de dados de treinamento. Devido à variância teórica do conjunto de dados de treinamento (lembramos que um conjunto de dados é uma amostra observada proveniente de uma verdadeira distribuição subjacente desconhecida - população), o modelo ajustado também está sujeito a variabilidade: se outro conjunto de dados tivesse sido observado, teríamos obtido um modelo diferente.

A ideia do bagging é, então, simples: queremos ajustar vários modelos independentes e "médias" de suas previsões para obter um modelo com menor variância. No entanto, na prática, não podemos ajustar modelos totalmente independentes porque exigiria muitos dados. Portanto, confiamos nas boas "propriedades aproximadas" das amostras bootstrap (representatividade e independência) para ajustar modelos que são quase independentes.

Suponha um conjunto $D$ de observações, e em cada iteração $i$, um conjunto de treinamento $D_i$ com $d$ observações é selecionado por meio de amostragem de linhas com reposição (ou seja, pode haver elementos repetitivos de diferentes observaçõs) de $D$ (ou seja, bootstrap). Em seguida, um modelo classificador $M_i$ é aprendido para cada conjunto de treinamento $D_i$. Cada classificador $M_i$ retorna sua previsão de classe. O classificador em conjunto $M^*$ conta os votos e atribui a classe com mais votos a $X$ (amostra desconhecida).

Passos de Implementação do Bagging

- Passo 1: Múltiplos subconjuntos são criados a partir do conjunto de dados original, selecionando observações com reposição.

- Passo 2: Um modelo base é criado para cada um desses subconjuntos.

- Passo 3: Cada modelo é aprendido em paralelo com cada conjunto de treinamento e de forma independente uns dos outros.

- Passo 4: As previsões finais são determinadas combinando as previsões de todos os modelos.

<figure>
<img src="../figures/Bagging.png" style="width:80%">
<figcaption align = "center"></figcaption>
</figure>

Existem várias maneiras possíveis de agregar os múltiplos modelos ajustados em paralelo. Para um problema de regressão, as saídas dos modelos individuais podem ser literalmente médias para obter a saída do modelo em conjunto.

$$
M^*(.) = \frac{1}{L}\sum_{i=1}^L M_i(.) 
$$

onde $M_i(.) \in \{M_1(.), M_2(.), \dots, M_L(.)\}$ e $L$ é o número de modelos. Para um problema de classificação, a classe produzida por cada modelo pode ser vista como um voto, e a classe que recebe a maioria dos votos é retornada pelo modelo em conjunto (isso é chamado de **voto duro**).

$$
M^*(.) = \arg\max_{c} \sum_{i=1}^L \mathbb{1}_{M_i(.) = c}
$$

Ainda para um problema de classificação, também podemos considerar as probabilidades de cada classe retornadas por todos os modelos, fazer a média dessas probabilidades e manter a classe com a maior média de probabilidade (isso é chamado de voto suave). As médias ou votos podem ser simples ou ponderados se pesos relevantes puderem ser utilizados.

Finalmente, vale mencionar que uma das grandes vantagens do bagging é que ele pode ser paralelizado. Como os diferentes modelos são ajustados independentemente uns dos outros, técnicas intensivas de paralelização podem ser usadas, se necessário.

## Random Forest - Floresta Aleatória

A floresta aleatória é um método de aprendizado em conjunto que combina o método de bagging com árvores de decisão. A ideia é simples: queremos ajustar várias árvores de decisão independentes e "médias" de suas previsões para obter um modelo com menor variância. No entanto, na prática, não podemos ajustar modelos totalmente independentes porque exigiria muitos dados. Portanto, confiamos nas boas "propriedades aproximadas" das amostras bootstrap (representatividade e independência) para ajustar modelos que são quase independentes.

A abordagem de floresta aleatória é um método de bagging em que **árvores profundas**, ajustadas em amostras de inicialização, são combinadas para produzir uma saída com menor variância. No entanto, as florestas aleatórias também utilizam outro truque para tornar as múltiplas árvores ajustadas um pouco menos correlacionadas entre si: ao **cultivar cada árvore**, em vez de apenas amostrar as observações no conjunto de dados para gerar uma amostra de inicialização, também amostramos features e mantemos apenas um subconjunto aleatório delas para construir a árvore.

<figure>
<img src="../figures/RandomForest.webp" style="width:50%">
<figcaption align = "center"></figcaption>
</figure>


Agora vamos ver como implementar o método de bagging e a floresta aleatória em Python. Vamos usar o mesmo conjunto de dados de Kyphosis que utilizamos no estudo de arvores de decisão para um problema de classificação binária.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

In [2]:
data_folder = '../data/'
df = pd.read_csv(data_folder + 'kyphosis.csv')

X = df.drop('Kyphosis',axis=1)
y = df['Kyphosis']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=41)

In [4]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [5]:
rfc_pred = rfc.predict(X_test)

In [6]:
print(confusion_matrix(y_test,rfc_pred))

[[18  4]
 [ 2  1]]


In [7]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

      absent       0.90      0.82      0.86        22
     present       0.20      0.33      0.25         3

    accuracy                           0.76        25
   macro avg       0.55      0.58      0.55        25
weighted avg       0.82      0.76      0.78        25



### Ajustando os hipermarametros - GridSearchCV

Agora vamos usar `GridSearchCV` para obter os melhores parâmetros para o modelo. Para isso, passaremos a instância `RandomFoestClassifier()` para o modelo e então ajustaremos o `GridSearchCV` usando os dados de treinamento para encontrar os melhores parâmetros.

Primeiro vamos criar um dicionário com os parâmetros que queremos testar. Para o `RandomForestClassifier()` vamos testar os seguintes parâmetros:

In [8]:
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [3, 6, 9], 
    'max_leaf_nodes': [3, 6, 9], 
} 

Começando a busca pelos melhores parâmetros. Isto pode demorar um pouco

In [9]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(RandomForestClassifier(), 
                           param_grid=param_grid) 
grid_search.fit(X_train, y_train) 
print(grid_search.best_estimator_) 

RandomForestClassifier(max_depth=6, max_features='log2', max_leaf_nodes=6,
                       n_estimators=50)


Agora podemos ajustar o modelo com os melhores parâmetros encontrados pelo `GridSearchCV`:

In [10]:
model_grid = RandomForestClassifier(max_depth=6, 
                                    max_features="log2", 
                                    max_leaf_nodes=6, 
                                    n_estimators=50) 
model_grid.fit(X_train, y_train) 
y_pred_grid = model_grid.predict(X_test) 
print(classification_report(y_pred_grid, y_test)) 

              precision    recall  f1-score   support

      absent       0.77      0.89      0.83        19
     present       0.33      0.17      0.22         6

    accuracy                           0.72        25
   macro avg       0.55      0.53      0.53        25
weighted avg       0.67      0.72      0.68        25



### Ajustando os hiperparametros - RandomizedSearchCV

Agora, vamos usar o `RandomizedSearchCV` para obter os melhores parâmetros para o modelo. Para isso, vamos passar uma instância de `RandomFoestClassifier()` para o modelo e, em seguida, ajustar o RandomizedSearchCV usando os dados de treinamento para encontrar os melhores parâmetros.

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(RandomForestClassifier(), 
                                   param_grid) 
random_search.fit(X_train, y_train) 
print(random_search.best_estimator_) 

RandomForestClassifier(max_depth=3, max_features=None, max_leaf_nodes=6,
                       n_estimators=150)


In [12]:

model_random = RandomForestClassifier(max_depth=3, 
                                      max_features=None, 
                                      max_leaf_nodes=6, 
                                      n_estimators=150) 
model_random.fit(X_train, y_train) 
y_pred_rand = model_random.predict(X_test) 
print(classification_report(y_pred_rand, y_test)) 

              precision    recall  f1-score   support

      absent       0.77      0.89      0.83        19
     present       0.33      0.17      0.22         6

    accuracy                           0.72        25
   macro avg       0.55      0.53      0.53        25
weighted avg       0.67      0.72      0.68        25



# Boosting

Os métodos de boosting funcionam no mesmo espírito dos métodos de bagging: construímos uma família de modelos que são agregados para obter um aprendiz forte que performa melhor. No entanto, ao contrário do bagging, que visa principalmente reduzir a variância, **o boosting é uma técnica que consiste em ajustar sequencialmente vários aprendizes fracos de uma maneira muito adaptativa: cada modelo na sequência é ajustado dando mais importância às observações no conjunto de dados que foram mal tratadas pelos modelos anteriores na sequência.** Intuitivamente, cada novo modelo concentra seus esforços nas observações mais difíceis de ajustar até agora, para que obtenhamos, no final do processo, um aprendiz forte com menor viés (mesmo que possamos observar que o boosting também pode ter o efeito de reduzir a variância). O boosting, assim como o bagging, pode ser usado tanto para problemas de regressão quanto para problemas de classificação.

Sendo principalmente focado em reduzir o viés, os modelos base que são frequentemente considerados para o boosting são modelos com baixa variância, mas alto viés. Por exemplo, se quisermos usar árvores como nossos modelos base, escolheremos na maioria das vezes árvores de decisão rasas com poucas profundidades. Outra razão importante que motiva o uso de modelos com baixa variância, mas alto viés, como aprendizes fracos para o boosting, é que esses modelos geralmente são menos computacionalmente caros de ajustar (poucos graus de liberdade quando parametrizados). De fato, como os cálculos para ajustar os diferentes modelos não podem ser feitos em paralelo (ao contrário do bagging), poderia se tornar muito caro ajustar sequencialmente vários modelos complexos.

Depois que os aprendizes fracos foram escolhidos, ainda precisamos definir como eles serão ajustados sequencialmente (que informações dos modelos anteriores levamos em consideração ao ajustar o modelo atual?) e como serão agregados (como agregamos o modelo atual aos anteriores?). Vamos discutir essas questões nas duas subseções seguintes, descrevendo mais especialmente dois algoritmos de boosting importantes: Adaboost e Gradient Boosting.



## Adaboost

No boost adaptativo (frequentemente chamado de “adaboost”), tentamos definir nosso modelo de conjunto como uma soma ponderada de $L$ aprendizes fracos


$$
M^*(.) = \sum_{i=1}^L \alpha_i M_i(.)
$$

onde $\alpha_i$ é o peso do modelo $M_i(.)$ na soma. A ideia é que, se pudermos encontrar os pesos certos, poderemos obter um modelo em conjunto com um desempenho muito melhor do que qualquer um dos modelos individuais. Para encontrar os pesos certos, precisamos definir uma função de perda que mede o quão bem nosso modelo em conjunto está se saindo em um conjunto de dados de treinamento. A função de perda mais comumente usada para o boosting é a função de perda exponencial, que é definida como

$$
L(y, M^*(x)) = \sum_{i=1}^N \exp(-y_i M^*(x_i))
$$

onde $y_i$ é a saída real e $M^*(x_i)$ é a saída prevista pelo modelo em conjunto para a observação $x_i$. A função de perda exponencial é uma função de perda convexa que penaliza muito mais os erros de classificação do que os erros de classificação corretos. De fato, se o modelo em conjunto prevê corretamente a classe de uma observação, o termo $\exp(-y_i M^*(x_i))$ é igual a 1, enquanto se o modelo em conjunto prevê incorretamente a classe de uma observação, o termo $\exp(-y_i M^*(x_i))$ é muito maior que 1. Portanto, a função de perda exponencial é muito sensível aos erros de classificação e, portanto, é uma boa função de perda para o boosting.

Agora que temos uma função de perda, podemos definir nosso algoritmo de boosting. O algoritmo de boosting mais comumente usado é o algoritmo Adaboost, que é um algoritmo de boosting sequencial que ajusta cada modelo na sequência de uma maneira muito adaptativa. O algoritmo Adaboost é definido da seguinte maneira:

1. Inicialize o conjunto de dados e atribua peso igual a cada ponto de dados.
2. Forneça isso como entrada para o modelo e identifique os pontos de dados classificados erroneamente.
3. Aumente o peso dos pontos de dados classificados erroneamente e diminua os pesos dos pontos de dados classificados corretamente. Em seguida, normalize os pesos de todos os pontos de dados.
4. Se (obteve os resultados desejados)
     Vá para a etapa 5
   Senão
     Vá para a etapa 2
5. Fim



<figure>
<img src="../figures/adaboost.webp" style="width:80%">
<figcaption align = "center"></figcaption>
</figure>

## Adaboost in scikit-learn
Agora vamos ver como implementar o método de Adaboost em Python usando a biblioteca `scikit-learn`. Vamos usar o mesmo conjunto de dados de Kyphosis  para um problema de classificação binária.

In [13]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=100)

ada.fit(X_train,y_train)

ada_pred = ada.predict(X_test)

print(classification_report(y_test,ada_pred))

              precision    recall  f1-score   support

      absent       0.95      0.86      0.90        22
     present       0.40      0.67      0.50         3

    accuracy                           0.84        25
   macro avg       0.68      0.77      0.70        25
weighted avg       0.88      0.84      0.86        25



### Ajustando os hipermarametros - GridSearchCV

In [14]:

# Definindo os parametros da malha
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Create an AdaBoost classifier
ada = AdaBoostClassifier()

# Create a GridSearchCV object
grid_search = GridSearchCV(ada, param_grid=param_grid)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print(grid_search.best_params_)


{'learning_rate': 1.0, 'n_estimators': 50}


In [15]:
ada_best = AdaBoostClassifier(learning_rate = 1, n_estimators=50) 
ada_best.fit(X_train, y_train) 
y_pred_best = ada_best.predict(X_test) 
print(classification_report(y_pred_best, y_test)) 

              precision    recall  f1-score   support

      absent       0.86      0.95      0.90        20
     present       0.67      0.40      0.50         5

    accuracy                           0.84        25
   macro avg       0.77      0.68      0.70        25
weighted avg       0.82      0.84      0.82        25



## Gradient Boosting

O Gradient Boosting é um dos algoritmos de aprendizado de máquina mais populares para conjuntos de dados tabulares. É poderoso o suficiente para encontrar qualquer relação não linear entre o alvo do seu modelo e as características, e possui grande utilidade que pode lidar com valores ausentes, outliers e valores categóricos de alta cardinalidade em suas características sem nenhum tratamento especial. Embora seja possível construir árvores de gradient boosting básicas usando bibliotecas populares como `XGBoost` ou `LightGBM` sem conhecer os detalhes do algoritmo, ainda é desejável entender como ele funciona ao ajustar hiperparâmetros, personalizar as funções de perda, etc., para obter uma melhor qualidade no seu modelo.

No Gradient Boosting, o modelo conjunto que tentamos construir também é uma soma ponderada de aprendizes fracos


$$
M^*(.) = \sum_{i=1}^L \alpha_i M_i(.)
$$

Assim como mencionamos para o Adaboost, encontrar o modelo ótimo sob essa forma é muito difícil, e uma abordagem iterativa é necessária. A principal diferença com o Adaboost está na definição do processo de otimização sequencial. Na verdade, o gradient boosting transforma o problema em um problema de descida de gradiente: em cada iteração, ajustamos um aprendiz fraco ao oposto do gradiente do erro de ajuste atual em relação ao modelo de conjunto atual. Em outras palavras, o modelo de conjunto atual é ajustado para minimizar o erro de ajuste atual. 

$$
M_L(.) = M_{L-1}(.) - \eta \nabla_{M_{L-1}} L(y, M_{L-1}(x))
$$

onde $\eta$ é a taxa de aprendizado (tamanho do passo) e $L$ é a função de perda (pseudo-resíduos do modelo anterior). A taxa de aprendizado é um hiperparâmetro que controla a rapidez com que o modelo em conjunto é ajustado para minimizar o erro de ajuste atual. Um valor baixo de $\eta$ significa que o modelo em conjunto é ajustado lentamente, enquanto um valor alto de $\eta$ significa que o modelo em conjunto é ajustado rapidamente. Um valor muito baixo de $\eta$ pode levar a um ajuste muito lento, enquanto um valor muito alto de $\eta$ pode levar a um ajuste muito rápido e a um ajuste excessivo. Portanto, a taxa de aprendizado é um hiperparâmetro importante que deve ser ajustado adequadamente.

Então, suponha que desejamos usar a técnica de gradient boosting com uma determinada família de modelos fracos. No início do algoritmo (primeiro modelo da sequência), os pseudo-resíduos são definidos como iguais aos valores das observações. Em seguida, repetimos $L$ vezes (para os $L$ modelos da sequência) as seguintes etapas:

1. Ajustar o melhor aprendiz fraco possível aos pseudo-resíduos (aproximar o oposto do gradiente em relação ao modelo forte atual).
2. Calcular o valor do tamanho de passo ótimo que define em que medida atualizamos o modelo em conjunto na direção do novo aprendiz fraco.
3. Atualizar o modelo em conjunto adicionando o novo aprendiz fraco multiplicado pelo tamanho de passo (dar um passo de descida de gradiente).
4. Calcular novos pseudo-resíduos que indicam, para cada observação, em qual direção gostaríamos de atualizar em seguida as previsões do modelo em conjunto.

Para uma discussão muito mais detalhada sobre o algoritmo de Gradient Boosting, consulte o excelente artigo de [All You Need to Know about Gradient Boosting Algorithm − Part 1. Regression](https://towardsdatascience.com/all-you-need-to-know-about-gradient-boosting-algorithm-part-1-regression-2520a34a502).



Há uma implementação do Gradient Boosting em Python na biblioteca `scikit-learn`. Porem, vamos usar a biblioteca `XGBoost` que é uma implementação mais eficiente do Gradient Boosting. Vamos usar o mesmo conjunto de dados de Kyphosis  para um problema de classificação binária.


In [25]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

y_test = y_test.replace('absent',0).replace('present',1)
y_train = y_train.replace('absent',0).replace('present',1)

xgb.fit(X_train,y_train)

xgb_pred = xgb.predict(X_test)

print(classification_report(y_test,xgb_pred))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        19
           1       0.60      0.50      0.55         6

    accuracy                           0.80        25
   macro avg       0.72      0.70      0.71        25
weighted avg       0.79      0.80      0.79        25



Tentando achar um bom valor para o hiperparâmetros usando o `GridSearchCV`:

In [38]:
xgb_classifier = XGBClassifier()

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
}

grid_search = GridSearchCV(xgb_classifier, param_grid=param_grid)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


Usando os melhores parâmetros encontrados pelo `GridSearchCV`:

In [27]:
xgb_best = XGBClassifier(**grid_search.best_params_)
xgb_best.fit(X_train, y_train)
y_pred_best = xgb_best.predict(X_test)
print(classification_report(y_pred_best, y_test))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85        21
           1       0.33      0.50      0.40         4

    accuracy                           0.76        25
   macro avg       0.61      0.65      0.62        25
weighted avg       0.80      0.76      0.78        25



Maximizando `f1-score`

In [28]:
xgb_classifier = XGBClassifier()

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
}

grid_search = GridSearchCV(xgb_classifier, param_grid=param_grid, scoring='f1')

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [29]:
xgb_best = XGBClassifier(**grid_search.best_params_)
xgb_best.fit(X_train, y_train)
y_pred_best = xgb_best.predict(X_test)
print(classification_report(y_pred_best, y_test))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85        21
           1       0.33      0.50      0.40         4

    accuracy                           0.76        25
   macro avg       0.61      0.65      0.62        25
weighted avg       0.80      0.76      0.78        25



# Stacking

Vamos brevemente discutir o método de aprendizado em conjunto chamado "stacking". 


O Stacking difere principalmente do Bagging e do Boosting em dois pontos essenciais. Primeiramente, o Stacking frequentemente considera aprendizes fracos heterogêneos (diferentes algoritmos de aprendizado são combinados), enquanto Bagging e Boosting consideram principalmente aprendizes fracos homogêneos. Em segundo lugar, o Stacking aprende a combinar os modelos base usando um meta-modelo, ao passo que Bagging e Boosting combinam aprendizes fracos seguindo algoritmos determinísticos.


Como mencionado anteriormente, a ideia por trás do Stacking é aprender vários aprendizes fracos diferentes e combiná-los treinando um meta-modelo para gerar previsões com base nas múltiplas previsões retornadas por esses modelos fracos. Para construir nosso modelo de Stacking, precisamos definir duas coisas: os L aprendizes que queremos ajustar e o meta-modelo que os combina.

Por exemplo, para um problema de classificação, podemos escolher como aprendizes fracos um classificador KNN, uma regressão logística e uma SVM, e decidir aprender um modelo neural como meta-modelo. Em seguida, a rede neural receberá como entradas as saídas dos nossos três aprendizes fracos e aprenderá a gerar previsões finais com base nelas.

Assumindo que desejamos ajustar um conjunto de Stacking composto por L aprendizes fracos, devemos seguir as etapas a seguir:

1. Dividir os dados de treinamento em dois conjuntos.
2. Escolher L aprendizes fracos e ajustá-los aos dados do primeiro conjunto.
3. Para cada um dos L aprendizes fracos, fazer previsões para as observações no segundo conjunto.
4. Ajustar o meta-modelo no segundo conjunto, usando as previsões feitas pelos aprendizes fracos como entradas.

## Stacking in scikit-learn

Agora vamos ver como implementar o método de Stacking em Python usando a biblioteca `scikit-learn`. Vamos usar o mesmo conjunto de dados de Kyphosis  para um problema de classificação binária.

In [30]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

base_models = [('rf_model', RandomForestClassifier()),
                ('gb_model', GradientBoostingClassifier())]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.76      0.86        25
           1       0.00      0.00      0.00         0

    accuracy                           0.76        25
   macro avg       0.50      0.38      0.43        25
weighted avg       1.00      0.76      0.86        25



/home/vahid/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vahid/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vahid/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dados Imbalanceados

Aqui a gente está vendo que o Stacking não é tão bom quanto o Gradient Boosting e o Adaboost. Mas, isso não significa que o Stacking não é um bom método de aprendizado em conjunto. O Stacking é um método muito poderoso, mas que precisa de um ajuste fino dos hiperparâmetros para obter um bom desempenho. Além disso, o Stacking é um método que pode ser muito caro computacionalmente, pois envolve o ajuste de vários modelos em paralelo e o ajuste de um meta-modelo. Portanto, o Stacking é um método que deve ser usado com cuidado, mas que pode ser muito poderoso se usado corretamente.

Como o nosso dataset é imbalenceado, vamos avisar os modelos que estamos usando que o dataset é imbalenceado. Neste caso ele vai dar mais peso para os dados mais raros. Para isso, vamos usar o parâmetro `class_weight='balanced'` na hora de criar os modelos.


In [31]:
df['Kyphosis'].value_counts()

Kyphosis
absent     64
present    17
Name: count, dtype: int64

In [32]:
base_models = [('rf_model', RandomForestClassifier(class_weight='balanced')),
                ('gb_model', GradientBoostingClassifier())]

meta_model = LogisticRegression(class_weight='balanced')

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83        17
           1       0.67      0.50      0.57         8

    accuracy                           0.76        25
   macro avg       0.73      0.69      0.70        25
weighted avg       0.75      0.76      0.75        25



Outro caminho é definir os pesos de cada classe manualmente. Para isso, vamos usar o parâmetro `class_weight={0: 1, 1: 5}` na hora de criar os modelos. Neste caso, estamos dizendo que a classe 0 tem peso 1 e a classe 1 tem peso 5. Ou seja, estamos dizendo que a classe 1 é 5 vezes mais importante que a classe 0.

In [35]:
class_weight = dict({0:1, 1:7})

base_models = [('rf_model', RandomForestClassifier(class_weight=class_weight)),
                ('gb_model', GradientBoostingClassifier())]

meta_model = LogisticRegression(class_weight=class_weight)

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.58      0.92      0.71        12
           1       0.83      0.38      0.53        13

    accuracy                           0.64        25
   macro avg       0.71      0.65      0.62        25
weighted avg       0.71      0.64      0.61        25



Outo caminho é usar a tecnica `SMOTE` para balancear o dataset. 

`SMOTE` (Synthetic Minority Over-sampling Technique) é uma técnica comumente utilizada para lidar com desequilíbrio de classes em conjuntos de dados. Em particular, o SMOTE é projetado para lidar com conjuntos de dados em que a classe minoritária (a classe menos prevalente) é significativamente menor em tamanho do que a classe majoritária.

A ideia central por trás do SMOTE é criar exemplos sintéticos da classe minoritária, introduzindo instâncias sintéticas entre as instâncias existentes dessa classe. Isso é feito gerando exemplos sintéticos ao longo das linhas que conectam instâncias minoritárias próximas no espaço de características. Isso ajuda a equilibrar a distribuição das classes no conjunto de dados.

A implementação básica do SMOTE envolve os seguintes passos:

- Selecionar uma instância da classe minoritária.
- Encontrar k vizinhos mais próximos dessa instância (tipicamente, k é definido pelo usuário).
- Escolher aleatoriamente um dos vizinhos e criar uma instância sintética ao longo da linha que conecta a instância original e o vizinho escolhido.

`Scikit-learn` fornece uma implementação do `SMOTE` na biblioteca `imbalanced-learn`, que é uma extensão do scikit-learn para lidar com conjuntos de dados desequilibrados. 

In [33]:
from imblearn.over_sampling import SMOTE

# aplicar o SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

base_models = [('rf_model', RandomForestClassifier()),
                ('gb_model', GradientBoostingClassifier())]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

stacking_model.fit(X_resampled, y_resampled)

y_pred = stacking_model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81        18
           1       0.50      0.43      0.46         7

    accuracy                           0.72        25
   macro avg       0.64      0.63      0.64        25
weighted avg       0.71      0.72      0.71        25



# Finalizando

Resumindo o que vimos até agora, podemos dizer que os métodos de aprendizado em conjunto são métodos que combinam vários algoritmos de aprendizado de máquina para obter um modelo preditivo mais poderoso. Existem dois tipos principais de métodos de aprendizado em conjunto: os métodos de "bagging", os métodos de "boosting" e o métodos de "stacking". Os métodos de bagging são métodos que visam reduzir a variância de um modelo em conjunto, combinando vários modelos em um modelo preditivo. O método de floresta aleatória é um método de bagging que combina o método de bagging com árvores de decisão. Os métodos de boosting são métodos que visam principalmente reduzir o viés de um modelo em conjunto, combinando vários modelos em um modelo preditivo. O método Adaboost é um método de boosting que combina o método de boosting com o método de descida de gradiente. O método de stacking é um método que combina vários algoritmos de aprendizado de máquina em um modelo preditivo, usando um meta-modelo para aprender como combinar os diferentes modelos.


| **Característica**                                | **Stacking**                                                     | **Bagging**                                                     | **Boosting**                                                     |
|----------------------------------------------------|------------------------------------------------------------------|------------------------------------------------------------------|------------------------------------------------------------------|
| **Tipo de Aprendizes Fracos**                      | Muitas vezes considera aprendizes fracos heterogêneos (algoritmos diferentes). | Principalmente considera aprendizes fracos homogêneos.          | Principalmente considera aprendizes fracos homogêneos.          |
| **Combinação de Aprendizes Fracos**                | Aprende a combinar aprendizes fracos usando um meta-modelo.      | Combina aprendizes fracos usando algoritmos determinísticos.    | Combina aprendizes fracos usando algoritmos determinísticos.    |
| **Abordagem de Aprendizado**                       | Combina diferentes algoritmos de aprendizado.                    | Treinamento paralelo de modelos independentes.                | Treinamento sequencial com foco em observações classificadas erroneamente. |
| **Objetivo**                                      | Combina a força de modelos diversos.                             | Reduz variância e overfitting.                                 | Reduz viés e underfitting.                                       |
| **Processo de Treinamento**                        | Treina vários aprendizes fracos e um meta-modelo.               | Treina vários modelos em paralelo.                             | Treina vários modelos sequencialmente, ajustando erros.        |
| **Exemplo**                                       | Generalização Empilhada (Stacking)                              | Floresta Aleatória (Random Forest)                            | AdaBoost, XGBoost                                                          |